In [1]:
%run uvms_dynamics.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4
floating_base found
floating_base found
floating_base found


In [2]:
ptheta = cs.vertcat(base_ss.sim_p, arm_ss.sim_p)
ptheta

SX([m, W, B, x_g, y_g, z_g, x_b, y_b, z_b, I_x, I_y, I_z, I_xz, X_du, Y_dv, Z_dw, K_dp, M_dq, N_dr, X_dq, Y_dp, M_du, K_dv, X_u, Y_v, Z_w, K_p, M_q, N_r, X_uu, Y_vv, Z_ww, K_pp, M_qq, N_rr, v_c_0, v_c_1, v_c_2, v_c_3, v_c_4, v_c_5, G_0, G_1, G_2, G_3, Ir3_14, Ir2_7, Ir1_7, Ir0_14, fw_static_0, fw_static_1, fw_static_2, fw_static_3, fw_viscous_0, fw_viscous_1, fw_viscous_2, fw_viscous_3, bw_static_0, bw_static_1, bw_static_2, bw_static_3, bw_viscous_0, bw_viscous_1, bw_viscous_2, bw_viscous_3])

In [3]:
tau, f_base , i_X_p_, i_X_0s_, Si_ = rig_dyn.get_inverse_dynamics_rnea(gravity=9.81, floating_base_id=tau_F, coupled=True)

floating_base found


In [5]:
tau.size()

(4, 1)

In [6]:
f_base.size()

(6, 1)